In [1]:
from bs4 import BeautifulSoup
import requests
import csv
from csv import writer

import pickle
import sys
import pandas as pd

In [2]:
url= "http://reports.ieso.ca/public/Adequacy2/PUB_Adequacy2.xml"

In [3]:
page = requests.get(url,allow_redirects=True)

In [4]:
soup = BeautifulSoup(page.text, 'lxml')
#lists = soup.find_all('div', class_="c1head")

/home/bhavya/miniconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [5]:
title= soup.find('doctitle').string
title

'Adequacy Report'

In [6]:
out_struc=pd.read_csv('adequacy_202208161044.csv')
print(out_struc.columns)
df= pd.DataFrame(columns=out_struc.columns)

Index(['mkt_date', 'mkt_he', 'dacp_flag', 'file_ver', 'created_at', 'ont_cap',
       'ont_ene', 'int_nuc_cap', 'int_nuc_out', 'int_nuc_off', 'int_nuc_sch',
       'int_gas_cap', 'int_gas_out', 'int_gas_off', 'int_gas_sch',
       'int_hyd_cap', 'int_hyd_out', 'int_hyd_for', 'int_hyd_off',
       'int_hyd_sch', 'int_win_cap', 'int_win_out', 'int_win_for',
       'int_win_sch', 'int_sol_cap', 'int_sol_out', 'int_sol_for',
       'int_sol_sch', 'int_bio_cap', 'int_bio_out', 'int_bio_off',
       'int_bio_sch', 'int_oth_cap', 'int_oth_out', 'int_oth_off',
       'int_oth_sch', 'int_tot_out', 'int_tot_off', 'int_tot_sch',
       'imp_man_off', 'imp_man_sch', 'imp_min_off', 'imp_min_sch',
       'imp_mic_off', 'imp_mic_sch', 'imp_new_off', 'imp_new_sch',
       'imp_que_off', 'imp_que_sch', 'imp_tot_off', 'imp_tot_sch',
       'imp_tot_est', 'imp_tot_cap', 'ont_bot_cap', 'ont_reg', 'ont_tot_sup',
       'ont_for_dem', 'ont_peak_dem', 'ont_avg_dem', 'ont_emb_wind',
       'ont_emb_sol', 'ont

In [7]:
# Helper Function to fetch the child columns
def fill_cap_list(element):
    xml_cap=element.find("capacities").find_all("capacity")
    cap=[]
    for element in xml_cap:
        if element.find("energymw")==None:
            cap.append(None)
            continue
        cap.append(element.find("energymw").text)
    return cap
def fill_out_list(element):
    xml_out=element.find("outages").find_all("outage")
    out=[]
    for element in xml_out:
        if element.find("energymw")==None:
            out.append(None)
            continue
        out.append(element.find("energymw").text)
    return out
def fill_off_list(element):
    xml_off=element.find("offers").find_all("offer")
    off=[]
    for element in xml_off:
        if element.find("energymw")==None:
            off.append(None)
            continue
        off.append(element.find("energymw").text)
    return off
def fill_sch_list(element):
    xml_sch=element.find("schedules").find_all("schedule")
    sch=[]
    for element in xml_sch:
        if element.find("energymw")==None:
            sch.append(None)
            continue
        sch.append(element.find("energymw").text)
    return sch
def fill_for_list(element):
    xml_arr=element.find("forecasts").find_all("forecast")
    arr=[]
    for element in xml_arr:
        if element.find("energymw")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymw").text)
    return arr
def fill_fore_list(element):
    xml_arr=element.find("forecastenergies").find_all("forecastenergy")
    arr=[]
    for element in xml_arr:
        if element.find("energymwhr")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymwhr").text)
    return arr
def fill_off_for_list(element):
    xml_arr=element.find("offerforecasts").find_all("offerforecast")
    arr=[]
    for element in xml_arr:
        if element.find("energymw")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymw").text)
    return arr
def fill_est_list(element):
    xml_arr=element.find("estimates").find_all("estimate")
    arr=[]
    for element in xml_arr:
        if element.find("energymw")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymw").text)
    return arr
def fill_bid_list(element):
    xml_arr=element.find("bids").find_all("bid")
    arr=[]
    for element in xml_arr:
        if element.find("energymw")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymw").text)
    return arr

In [8]:
# Ontario Capacity
xml_cap=soup.find("capacities").find_all("capacity")
ont_cap=[]
for element in xml_cap:
    if element.find("energymw")==None:
        ont_cap.append(None)
        continue
    ont_cap.append(element.find("energymw").text)
df['ont_cap']=ont_cap

# Ontario Energy
xml_ene=soup.find("energies").find_all("energy")
ont_ene=[]
for element in xml_ene:
    ont_ene.append(element.find("energymwhr").text)
df['ont_ene']=ont_ene

In [9]:
# Internal Resources
xml_int=soup.find("internalresources").find_all("internalresource")
for int_element in xml_int:
    if int_element.find("fueltype").text=='Nuclear':
        df['int_nuc_cap']=fill_cap_list(int_element)
        df['int_nuc_out']=fill_out_list(int_element)
        df['int_nuc_off']=fill_off_list(int_element)
        df['int_nuc_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Gas':
        df['int_gas_cap']=fill_cap_list(int_element)
        df['int_gas_out']=fill_out_list(int_element)
        df['int_gas_off']=fill_off_list(int_element)
        df['int_gas_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Hydro':
        df['int_hyd_cap']=fill_cap_list(int_element)
        df['int_hyd_out']=fill_out_list(int_element)
        df['int_hyd_for']=fill_fore_list(int_element)
        df['int_hyd_off']=fill_off_list(int_element)
        df['int_hyd_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Wind':
        df['int_win_cap']=fill_cap_list(int_element)
        df['int_win_out']=fill_out_list(int_element)
        df['int_win_for']=fill_for_list(int_element)
        df['int_win_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Solar':
        df['int_sol_cap']=fill_cap_list(int_element)
        df['int_sol_out']=fill_out_list(int_element)
        df['int_sol_for']=fill_for_list(int_element)
        df['int_sol_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Biofuel':
        df['int_bio_cap']=fill_cap_list(int_element)
        df['int_bio_out']=fill_out_list(int_element)
        df['int_bio_off']=fill_off_list(int_element)
        df['int_bio_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Other':
        df['int_oth_cap']=fill_cap_list(int_element)
        df['int_oth_out']=fill_out_list(int_element)
        df['int_oth_off']=fill_off_for_list(int_element)
        df['int_oth_sch']=fill_sch_list(int_element)
    else:
        print('Unknown Fueltype')
        
# Total Internal Resources
xml_tot_int=soup.find("internalresources").find("totalinternalresources")
df['int_tot_out']=fill_out_list(xml_tot_int)
df['int_tot_off']=fill_off_for_list(xml_tot_int)
df['int_tot_sch']=fill_sch_list(xml_tot_int)

In [10]:
# Imports
xml_imp=soup.find("zonalimports").find_all("zonalimport")
for imp_element in xml_imp:
    if imp_element.find("zonename").text=='Manitoba':
        df['imp_man_off']=fill_off_list(imp_element)
        df['imp_man_sch']=fill_sch_list(imp_element)
    elif imp_element.find("zonename").text=='Minnesota':
        df['imp_min_off']=fill_off_list(imp_element)
        df['imp_min_sch']=fill_sch_list(imp_element)
    elif imp_element.find("zonename").text=='Michigan':
        df['imp_mic_off']=fill_off_list(imp_element)
        df['imp_mic_sch']=fill_sch_list(imp_element)
    elif imp_element.find("zonename").text=='New York':
        df['imp_new_off']=fill_off_list(imp_element)
        df['imp_new_sch']=fill_sch_list(imp_element)
    elif imp_element.find("zonename").text=='Quebec':
        df['imp_que_off']=fill_off_list(imp_element)
        df['imp_que_sch']=fill_sch_list(imp_element)
    else:
        print('Unknown Fueltype')
        
# Total Imports
xml_tot_imp=soup.find("zonalimports").find("totalimports")
df['imp_tot_off']=fill_off_list(xml_tot_imp)
df['imp_tot_sch']=fill_sch_list(xml_tot_imp)
df['imp_tot_est']=fill_est_list(xml_tot_imp)
df['imp_tot_cap']=fill_cap_list(xml_tot_imp)

In [11]:
# Bottled Capacity
xml_bot_cap=soup.find("bottledcapacities").find_all("capacity")
ont_bot_cap=[]
for element in xml_bot_cap:
    if element.find("energymw")==None:
        ont_bot_cap.append(None)
        continue
    ont_bot_cap.append(element.find("energymw").text)
df['ont_bot_cap']=ont_bot_cap

# Regulation
xml_reg=soup.find("regulations").find_all("regulation")
ont_reg=[]
for element in xml_reg:
    if element.find("energymw")==None:
        ont_reg.append(None)
        continue
    ont_reg.append(element.find("energymw").text)
df['ont_reg']=ont_reg

# Total Supply
xml_tot_sup=soup.find("totalsupplies").find_all("supply")
ont_tot_sup=[]
for element in xml_tot_sup:
    if element.find("energymw")==None:
        ont_tot_sup.append(None)
        continue
    ont_tot_sup.append(element.find("energymw").text)
df['ont_tot_sup']=ont_tot_sup

In [12]:
# Ontario Forecast Demand
xml_for_dem=soup.find("forecastontdemand").find_all("demand")
ont_for_dem=[]
for element in xml_for_dem:
    if element.find("energymw")==None:
        ont_for_dem.append(None)
        continue
    ont_for_dem.append(element.find("energymw").text)
df['ont_for_dem']=ont_for_dem

# Ontario Peak Demand
xml_peak_dem=soup.find("peakdemand").find_all("demand")
ont_peak_dem=[]
for element in xml_peak_dem:
    if element.find("energymw")==None:
        ont_peak_dem.append(None)
        continue
    ont_peak_dem.append(element.find("energymw").text)
df['ont_peak_dem']=ont_peak_dem

# Ontario Average Demand
xml_avg_dem=soup.find("averagedemand").find_all("demand")
ont_avg_dem=[]
for element in xml_avg_dem:
    if element.find("energymw")==None:
        ont_avg_dem.append(None)
        continue
    ont_avg_dem.append(element.find("energymw").text)
df['ont_avg_dem']=ont_avg_dem

# Ontario Wind Embedded
xml_emb_wind=soup.find("windembedded").find_all("embedded")
ont_emb_wind=[]
for element in xml_emb_wind:
    if element.find("energymw")==None:
        ont_emb_wind.append(None)
        continue
    ont_emb_wind.append(element.find("energymw").text)
df['ont_emb_wind']=ont_emb_wind

# Ontario Solar Embedded
xml_emb_sol=soup.find("solarembedded").find_all("embedded")
ont_emb_sol=[]
for element in xml_emb_sol:
    if element.find("energymw")==None:
        ont_emb_sol.append(None)
        continue
    ont_emb_sol.append(element.find("energymw").text)
df['ont_emb_sol']=ont_emb_sol

# Dispatchable Load Capacity
xml_arr=soup.find("dispatchableload").find("capacities").find_all("capacity")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_disp_cap']=arr

# Dispatchable Load Bids
xml_arr=soup.find("dispatchableload").find("bidforecasts").find_all("bidforecast")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_disp_bid']=arr

# Dispatchable Load On
xml_arr=soup.find("dispatchableload").find("scheduledon").find_all("schedule")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_disp_on']=arr

# Dispatchable Load Off
xml_arr=soup.find("dispatchableload").find("scheduledoff").find_all("schedule")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_disp_off']=arr


# Hourly Demand Response Bids
xml_arr=soup.find("hourlydemandresponse").find("bids").find_all("bid")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_hdr_bid']=arr

# Hourly Demand Response Scheduled
xml_arr=soup.find("hourlydemandresponse").find("schedules").find_all("schedule")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_hdr_sch']=arr

# Hourly Demand Response Curtailed
xml_arr=soup.find("hourlydemandresponse").find("curtailed").find_all("curtail")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['ont_hdr_cur']=arr

In [13]:
# Exports
xml_exp=soup.find("zonalexports").find_all("zonalexport")
for exp_element in xml_exp:
    if exp_element.find("zonename").text=='Manitoba':
        df['exp_man_bid']=fill_bid_list(exp_element)
        df['exp_man_sch']=fill_sch_list(exp_element)
    elif exp_element.find("zonename").text=='Minnesota':
        df['exp_min_bid']=fill_bid_list(exp_element)
        df['exp_min_sch']=fill_sch_list(exp_element)
    elif exp_element.find("zonename").text=='Michigan':
        df['exp_mic_bid']=fill_bid_list(exp_element)
        df['exp_mic_sch']=fill_sch_list(exp_element)
    elif exp_element.find("zonename").text=='New York':
        df['exp_new_bid']=fill_bid_list(exp_element)
        df['exp_new_sch']=fill_sch_list(exp_element)
    elif exp_element.find("zonename").text=='Quebec':
        df['exp_que_bid']=fill_bid_list(exp_element)
        df['exp_que_sch']=fill_sch_list(exp_element)
    else:
        print('Unknown Province')
        
# Total Exports
xml_tot_exp=soup.find("zonalexports").find("totalexports")
df['exp_tot_bid']=fill_bid_list(xml_tot_exp)
df['exp_tot_sch']=fill_sch_list(xml_tot_exp)
df['exp_tot_cap']=fill_cap_list(xml_tot_exp)

In [14]:
# Generation reserve holdback total
xml_arr=soup.find("generationreserveholdback").find("totalorreserve").find_all("orreserve")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['grh_tot']=arr

# Generation reserve holdback min_10n
xml_arr=soup.find("generationreserveholdback").find("min10minor").find_all("min10or")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['grh_min_10n']=arr

# Generation reserve holdback min_10s
xml_arr=soup.find("generationreserveholdback").find("min10minspinor").find_all("min10spinor")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['grh_min_10s']=arr

# Generation reserve holdback LFU
xml_arr=soup.find("generationreserveholdback").find("loadforecastuncertainties").find_all("uncertainty")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['grh_lfu']=arr

# Generation reserve holdback Allowances
xml_arr=soup.find("generationreserveholdback").find("contingencyallowances").find_all("allowance")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['grh_add']=arr

In [15]:
# Total Requiremnts
xml_arr=soup.find("totalrequirements").find_all("requirement")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['total_req']=arr

# Capacity Excess/Shortfall
xml_arr=soup.find("excesscapacities").find_all("capacity")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['cap_excess']=arr

# Energy Excess/Shortfall (MWhr)
xml_arr=soup.find("excessenergies").find_all("energy")
arr=[]
for element in xml_arr:
    if element.find("energymwhr")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymwhr").text)
df['energy_excess']=arr

# Offered Capacity Excess/Shortfall
xml_arr=soup.find("excessofferedcapacities").find_all("capacity")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['offered_cap_excess']=arr

# Resources Not Scheduled
xml_arr=soup.find("unscheduledresources").find_all("unscheduledresource")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['res_not_sch']=arr

# Imports Not Scheduled
xml_arr=soup.find("unscheduledimports").find_all("unscheduledimport")
arr=[]
for element in xml_arr:
    if element.find("energymw")==None:
        arr.append(None)
        continue
    arr.append(element.find("energymw").text)
df['imp_not_sch']=arr


In [21]:
# Filling Created_at, Marketing_date, Hour_ending columns

if soup.find("createdat")!=None:
    xml_cdate=soup.find("createdat").text
    df['created_at'].fillna(xml_cdate,inplace=True)

if soup.find("deliverydate")!=None:
    xml_mdate=soup.find("deliverydate").text
    df['mkt_date'].fillna(xml_mdate,inplace=True)

df['mkt_he']=list(df.index + 1)

In [30]:
pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)
df.head(24)

,mkt_date,mkt_he,dacp_flag,file_ver,created_at,ont_cap,ont_ene,int_nuc_cap,int_nuc_out,int_nuc_off,int_nuc_sch,int_gas_cap,int_gas_out,int_gas_off,int_gas_sch,int_hyd_cap,int_hyd_out,int_hyd_for,int_hyd_off,int_hyd_sch,int_win_cap,int_win_out,int_win_for,int_win_sch,int_sol_cap,int_sol_out,int_sol_for,int_sol_sch,int_bio_cap,int_bio_out,int_bio_off,int_bio_sch,int_oth_cap,int_oth_out,int_oth_off,int_oth_sch,int_tot_out,int_tot_off,int_tot_sch,imp_man_off,imp_man_sch,imp_min_off,imp_min_sch,imp_mic_off,imp_mic_sch,imp_new_off,imp_new_sch,imp_que_off,imp_que_sch,imp_tot_off,imp_tot_sch,imp_tot_est,imp_tot_cap,ont_bot_cap,ont_reg,ont_tot_sup,ont_for_dem,ont_peak_dem,ont_avg_dem,ont_emb_wind,ont_emb_sol,ont_disp_cap,ont_disp_bid,ont_disp_on,ont_disp_off,ont_hdr_bid,ont_hdr_sch,ont_hdr_cur,exp_man_bid,exp_man_sch,exp_min_bid,exp_min_sch,exp_mic_bid,exp_mic_sch,exp_new_bid,exp_new_sch,exp_que_bid,exp_que_sch,exp_tot_bid,exp_tot_sch,exp_tot_cap,grh_tot,grh_min_10n,grh_min_10s,grh_lfu,grh_add,total_req,cap_excess,energy_excess,offered_cap_excess,res_not_sch,imp_not_sch
0,2022-09-21,1,NaN,NaN,2022-08-18T15:36:43,37466,22207,13081,4100,None,None,10679,3630,None,None,7954,2087,3929,None,None,4943,3732,1211,None,478,478,0,None,315,293,None,None,17,0,None,None,14761,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1000,100,23704,13779,None,13779,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1418,945,236,0,161,15233,8471,8553,None,None,None
1,2022-09-21,2,NaN,NaN,2022-08-18T15:36:43,37466,21969,13081,4100,None,None,10679,3630,None,None,7954,2087,3691,None,None,4943,3732,1211,None,478,478,0,None,315,293,None,None,17,0,None,None,14761,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1000,100,23704,13340,None,13340,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1418,945,236,0,161,14794,8910,8754,None,None,None
2,2022-09-21,3,NaN,NaN,2022-08-18T15:36:43,37466,21935,13081,4100,None,None,10679,3630,None,None,7954,2087,3657,None,None,4943,3732,1211,None,478,478,0,None,315,293,None,None,17,0,None,None,14761,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1000,100,23704,13098,None,13098,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1418,945,236,0,161,14552,9152,8962,None,None,None
3,2022-09-21,4,NaN,NaN,2022-08-18T15:36:43,37466,21955,13081,4100,None,None,10679,3630,None,None,7954,2087,3677,None,None,4943,3732,1211,None,478,478,0,None,315,293,None,None,17,0,None,None,14761,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1000,100,23704,13118,None,13118,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1418,945,236,0,161,14572,9132,8962,None,None,None
4,2022-09-21,5,NaN,NaN,2022-08-18T15:36:43,37466,21978,13081,4100,None,None,10679,3630,None,None,7954,2087,3700,None,None,4943,3732,1211,None,478,478,0,None,315,293,None,None,17,0,None,None,14761,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1000,100,23704,13699,None,13699,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1418,945,236,0,161,15152,8552,8405,None,None,None
5,2022-09-21,6,NaN,NaN,2022-08-18T15:36:43,37466,22134,13081,4100,None,None,10679,3630,None,None,7954,2087,3856,None,None,4943,3732,1211,None,478,478,0,None,315,293,None,None,17,0,None,None,14761,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2000,100,1000,100,23704,15197,None,15197,None,None,513,125,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1418,945,236,0,161,16651,7053,7062,None,None,None
6,2022-09-21,7,NaN,NaN,2022-08-18T15:36:43,37466,22317,13081,4100,None,None,10679,3630,None,None,7954,2091,4039,None,None,4943,3732,1211,None,478,478,0,None,315,293,Non